In [ ]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib inline 
import sys

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

In [ ]:
def Conv2d(input, output_dim=64, kernel=(5, 5), strides=(2, 2), stddev=0.2, name='conv_2d'):

    with tf.variable_scope(name):
        W = tf.get_variable('Conv2d_W', [kernel[0], kernel[1], input.get_shape()[-1], output_dim],
                             initializer=tf.truncated_normal_initializer(stddev=stddev))
        b = tf.get_variable('Conv2d_b', [output_dim], initializer=tf.zeros_initializer())
        
        return tf.nn.conv2d(input, W, strides=[1, strides[0], strides[1], 1], padding='SAME') + b



def Dense(input, output_dim, stddev=0.02, name='dense'):
    
    with tf.variable_scope(name):
        
        shape = input.get_shape()
        W = tf.get_variable('Dense_W', [shape[1], output_dim], tf.float32,
                            tf.random_normal_initializer(stddev=stddev))
        b = tf.get_variable('Dense_b', [output_dim],
                            initializer=tf.zeros_initializer())
        
        return tf.matmul(input, W) + b

In [ ]:
def Discriminator(X, reuse=False, name='D'):

    with tf.variable_scope(name, reuse=reuse):
        
        print('\nDiscriminator vars\n')
        
        
        X_reshaped = tf.reshape(X, [-1, 28, 28, 1])
        D_conv1 = Conv2d(X_reshaped, output_dim=64, name='d_conv1')
        print('D_conv1', D_conv1.get_shape())
        
        
        D_h1=tf.nn.leaky_relu(D_conv1)
        print('D_h1', D_h1.get_shape())
        
        
        D_conv2 = Conv2d(D_h1, output_dim=128, name='d_conv2')
        print('D_conv2', D_conv2.get_shape())
        
        
        D_h2=tf.nn.leaky_relu(D_conv2)
        D_r2 = tf.reshape(D_h2, [-1, 256])
        print('D_r2', D_r2.get_shape())
        
        
        D_h3=tf.nn.leaky_relu(D_r2)
        D_h4 = tf.nn.dropout(D_h3, 0.5)
        print('D_h4', D_h4.get_shape())
        
        
        D_h5 = Dense(D_h4, output_dim=1, name='d_h5') 
        print('D_h5', D_h5.get_shape())
        
        
        return tf.nn.sigmoid(D_h5)

In [ ]:
def Generator(Z, name='G'):

    with tf.variable_scope(name):
        
        print('\nGenerator vars\n')
        G_1 = Dense(Z, output_dim=1024, name='g_1') 
        print('G_1', G_1.get_shape())
        
        
        G_bn1 = tf.contrib.layers.batch_norm(inputs = G_1, scale=True, scope="g_bn1")
        print('G_bn1',G_bn1.get_shape())
        
        
        G_h1 = tf.nn.relu(G_bn1)
        print('G_h1',G_h1.get_shape())
        
        
        G_2 = Dense(G_h1, output_dim=7*7*128, name='g_2') # [-1, 7*7*128]
        print('G_2',G_2.get_shape())
        
      
        G_bn2 = tf.contrib.layers.batch_norm(inputs = G_2, scale=True, scope="g_bn2")
        print('G_bn2',G_bn2.get_shape())
        
        G_h2 = tf.nn.relu(G_bn2)
        print('G_h2',G_h2.get_shape())
        G_r2 = tf.reshape(G_h2, [-1, 7, 7, 128])
        print('G_r2',G_r2.get_shape())
        

        G_conv3 = tf.layers.conv2d_transpose(G_r2, 64, (5,5), (2,2), padding='same', name='g_conv3',
                                             activation=None, 
                                             kernel_initializer=tf.truncated_normal_initializer(stddev=0.2), 
                                             bias_initializer=tf.constant_initializer(.1))
        print('G_conv3',G_conv3.get_shape())
        
        
        G_conv3 = tf.contrib.layers.batch_norm(inputs = G_conv3, scale=True, scope="g_bn3")
        print('G_conv3',G_conv3.get_shape())
        
        
        G_conv3 = tf.nn.relu(G_conv3)
        print('G_conv3',G_conv3.get_shape())
        
        
        
        G_conv4 = tf.layers.conv2d_transpose(G_conv3, 1, (5,5), (2,2), padding='same', name='g_conv4',
                                             activation=None, 
                                             kernel_initializer=tf.truncated_normal_initializer(stddev=0.2), 
                                             bias_initializer=tf.constant_initializer(.1))
        
        print('G_conv4',G_conv4.get_shape())
        
        
        
        G_r4 = tf.reshape(G_conv4, [-1, 784])
        print('G_r4',G_r4.get_shape())
        
        
        return tf.nn.sigmoid(G_r4)

In [ ]:
X = tf.placeholder(tf.float32, shape=[None, 28*28])
Z = tf.placeholder(tf.float32, shape=[None, 100])

G = Generator(Z, 'G')
D_loss_real = Discriminator(X, False, 'D')
D_loss_fake = Discriminator(G, True, 'D')

# discriminator loss
D_loss = -tf.reduce_mean(tf.log(D_loss_real) - tf.log(D_loss_fake)) 
# generator loss
G_loss = -tf.reduce_mean(tf.log(D_loss_fake)) 

tvars = tf.trainable_variables()
d_vars = [v for v in tvars if v.name.startswith('D/')]
g_vars = [v for v in tvars if v.name.startswith('G/')]

optimizerD = tf.train.AdamOptimizer(learning_rate=1e-4, beta1=0.1).minimize(D_loss, var_list=d_vars)
optimizerG = tf.train.AdamOptimizer(learning_rate=2e-4, beta1=0.3).minimize(G_loss, var_list=g_vars)

In [ ]:
def plot(samples, D_loss, G_loss, epoch, calc_number):

    fig = plt.figure(figsize=(10, 5))        

    gs = gridspec.GridSpec(4, 8)
    gs.update(wspace=0.05, hspace=0.05)
    
    # plot losses
    ax = plt.subplot(gs[:, 4:])
    ax.plot(D_loss, label="discriminator's loss", color='b')
    ax.plot(G_loss, label="generator's loss", color='r')
    ax.set_xlim([0, calc_number])
    ax.yaxis.tick_right()
    ax.legend()

    # create new images
    for i, sample in enumerate(samples):
        
        # let's draw 16 pictures (4 by 4 grid). If we want more - change it
        if i == 16:
            break
            
        ax = plt.subplot(gs[i % 4, int(i / 4)])
        plt.axis('off')
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28, 28), cmap='Greys_r')

    plt.savefig('./output_by_epochs/' + str(epoch + 1) + '.png')
    plt.close()  

In [ ]:
BATCH_SIZE = 64
EPOCHS = 40
#EPOCHS = 3
D_losses = []
G_losses = []
iterations = int(mnist.train.images.shape[0] / BATCH_SIZE)
# if we wanna draw plots with losses relatively the whole process-> we have to know the full number of calculations
calc_number = EPOCHS * iterations

    
with tf.Session() as sess:

    sess.run(tf.global_variables_initializer())
    

    for e in range(EPOCHS):
        for i in range(iterations):
            
            x, _ = mnist.train.next_batch(BATCH_SIZE)
            z = np.random.uniform(0.0, 1.0, size=[BATCH_SIZE, 100])
            _, D_loss_curr = sess.run([optimizerD, D_loss], {X: x, Z: z})
            D_losses.append(D_loss_curr)
            
            
            #z = np.random.uniform(0.0, 1.0, size=[BATCH_SIZE, 100])
            _, G_loss_curr = sess.run([optimizerG, G_loss], {Z: z})
            G_losses.append(G_loss_curr)
            
            # good looking output =)
            sys.stdout.write("\r%d / %d epochs, %d / %d iterations: D_loss = %f, G_loss = %f"
                             %(e, EPOCHS-1, i, iterations-1, D_loss_curr, G_loss_curr))
            sys.stdout.flush()
            
        # each epoch generate images and create an output "set of pictures + losses"
        data = sess.run(G, {Z: z})
        plot(data, D_losses, G_losses, e, calc_number) 